In [1]:
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("../RD-systems-and-test-benches/utils")
import model_fsolve as modf
import model_fsolve_v2 as modf2

import scipy.optimize as sco
import scipy.interpolate as sci

import hx_hydraulic as hxhy
import from_excel as fe
import data_plots as dplt

import fluids as fds
from CoolProp.CoolProp import PropsSI

import openpyxl as opxl
from sklearn.ensemble import RandomForestRegressor

import seaborn as sns
import os
import plotly.graph_objects as go

import copy


In [2]:
import repo_config as rconfig

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.ticker import FuncFormatter
import scienceplots
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
plt.style.use(['science'])

def to_percent(y, _):
    return f"{y * 100:.0f}\%"

In [3]:
path = r"G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\Inputs"
file_name = "\MICOE.xlsx"
hw, par, cond = fe.initialize(path, file_name)
tabl, res, PL, residuals = modf.PL_fsolve(par, cond) 

fluid = {'name':'MPG','glycol_rate':0.4}

In [4]:
Vdot_range = [30, 50, 100, 150, 272, 478, 652, 856, 1094]

In [5]:
glycol_range = [0.2,0.25,0.3,0.35,0.4]

glycol_dict = {}

for i, glycol_rate in enumerate(glycol_range):

    res_dict = []

    fe.change_fluid(cond, {'name' : 'MPG', 'glycol_rate' : glycol_rate})

    for Vdot in Vdot_range:
        cond_local = copy.deepcopy(cond)
        fe.change_Vdot(cond_local, Vdot)

        tabl, res, PL, residuals = modf.PL_fsolve(par, cond_local)

        res_dict.append({
                        'Vdot' : Vdot,  
                        'tabl' : copy.deepcopy(tabl),
                        'PL' : copy.deepcopy(res),
                        'tabl_PL' : PL,
                        'residuals' : copy.deepcopy(residuals)
                        }
                        )
        
    glycol_dict[glycol_rate] = copy.deepcopy(res_dict)

In [6]:
res_df = pd.DataFrame()

for i, glycol_rate in enumerate(glycol_range):

    res_dict = glycol_dict[glycol_rate]
    keys_to_include = ['Vdot','PL']

    temp = pd.DataFrame([{key: d[key] for key in keys_to_include} for d in res_dict])
    res_df['Vdot'] = temp['Vdot']
    res_df['PL glycol = ' + str(glycol_rate)] = temp['PL']

In [ ]:
res_df

In [8]:
res_df['PL_measure'] = [np.nan, np.nan, np.nan, np.nan, 485,977,1336,1941,2800]
# res_df['relative error'] = (res_df['PL'] - res_df['PL_measure'])/res_df['PL_measure']
# res_df

In [ ]:
glycol_dict[0.3][-1]['tabl_PL']

In [ ]:
res_df

In [10]:
res_df['error'] = res_df['PL glycol = 0.3'] - res_df['PL_measure']

In [14]:
res_df['relative error'] = res_df['error']/res_df['PL_measure']

In [ ]:
res_df['error'].abs().mean()/res_df['PL_measure'].mean()

In [ ]:
colors = ['olive', 'green', 'blue', 'orange', 'red', 'grey']
colors_lighter = [rconfig.get_hex_code(color, 70) for color in colors]
colors = [rconfig.get_hex_code(color, 100) for color in colors]
linestyles = ['-', '--',  (5, (10, 3)), '-.', (0, (1, 1)), (0, (3, 5, 1, 5))]
markers = ['o', 's', 'd', 'X', 'v', '^']

# Main plot: plotting L_f0_range on x-axis
fig, ax1 = plt.subplots(figsize=(rconfig.wcol_in, rconfig.hfig(0.3, unit="in")))

# Create the plot
x = res_df['Vdot']
y_min = res_df['PL glycol = 0.2']
y_mid = res_df['PL glycol = 0.3']
y_max = res_df['PL glycol = 0.4']

# Plot lines for different glycol concentrations
plt.plot(x, y_min, color=colors[0], linestyle=linestyles[1], label='Model w/ MPG 20%')
plt.plot(x, y_mid, color=colors[2], linestyle=linestyles[3], label='Model w/ MPG 30%')
plt.plot(x, y_max, color=colors[1], linestyle=linestyles[2], label='Model w/ MPG 40%')

# Fill the area between y_min and y_max
plt.fill_between(x, y_min, y_max, color=colors_lighter[0], alpha=0.2)

# Scatter plot for the measurement points
plt.scatter(x, res_df['PL_measure'], label='Measure', color=colors[-1], marker='o')

# Set labels and grid
ax1.set_xlabel('Vdot [L/h]')
ax1.set_ylabel('PL [kPa]')
ax1.grid()

# Set axis limits
ax1.set_xlim([0, 1200])
ax1.set_ylim([0, 3500])

# Create the first row of the legend (for the lines)
first_legend = fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.), ncol=3, frameon=False,
                          labels=['Model w/ MPG 20\%', 'Model w/ MPG 30\%', 'Model w/ MPG 40\%'],
                          handles=[ax1.lines[0], ax1.lines[1], ax1.lines[2]])

# Create the second row of the legend (for the scatter plot)
second_legend = fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.08), ncol=1, frameon=False,
                           labels=['Measure'], handles=[ax1.collections[1]])

# Add the first legend back to the axes (to avoid being overwritten)
ax1.add_artist(first_legend)

# Show the plot
plt.show()

In [ ]:
ax1.collections

In [71]:
fig.savefig(r'G:\Mon Drive\GitHub\VDE_thesis\image\Chapter_PL\PL-2.3-MICOE-fit.png', dpi=600)

In [ ]:
fig, ax1 = plt.subplots(figsize=(rconfig.wcol_in, rconfig.hfig(0.3, unit="in")))

for i in range(len(res_dict)):
    if i >= 4:
        tabl = res_dict[i]['tabl']
        tabl['qx_norm'] = tabl['qx'] / (tabl['qx'].sum() / par['N'])
        ax1.plot(tabl.index, tabl['qx_norm'], 
                 label='Vdot = ' + str(res_dict[i]['Vdot']), 
                 color=colors[i-4], 
                 linestyle=linestyles[i-4])

# Gather handles and labels for the legend
handles, labels = ax1.get_legend_handles_labels()

# Create the first row of the legend (for the first half of the labels)
first_row_legend = fig.legend(handles[:3], labels[:3], 
                              loc='upper center', 
                              bbox_to_anchor=(0.5, 0.), 
                              ncol=3, 
                              frameon=False,
                              handlelength=3)  # Increase handle length

# Create the second row of the legend (for the second half of the labels)
second_row_legend = fig.legend(handles[3:], labels[3:], 
                               loc='upper center', 
                               bbox_to_anchor=(0.5, -0.07), 
                               ncol=3, 
                               frameon=False,
                               handlelength=3)  # Increase handle length

# Add the first row back to the axes to avoid it being overwritten
ax1.add_artist(first_row_legend)

ax1.set_xlabel('Tube number')
ax1.set_ylabel('Normalized flow rate [-]')
ax1.grid()

ax1.set_xlim([0, 23])
# Show the plot
plt.show()

In [93]:
fig.savefig(r'G:\Mon Drive\GitHub\VDE_thesis\image\Chapter_PL\PL-2.3-MICOE-distribution.png', dpi=600)